In [1]:
import pandas as pd
df = pd.read_csv('imdb_indonesian_movies_2.csv')
df = df.drop(columns=['judul_film'])

In [2]:
df.head()

,ringkasan_sinopsis,genre
0,Raden Mas Said putra sulung Tumenggung Wilarik...,Drama
1,Soe Hok Gie adalah seorang aktivis yang hidup ...,Drama
2,Guru Bangsa Tjokroaminoto menceritakan tentang...,Drama
3,POL menceritakan kisah hidup yang luar biasa d...,Drama
4,Perjalanan pahlawan Indonesia KH Ahmad Dahlan ...,Drama


In [3]:
category = pd.get_dummies(df.genre)
df_new = pd.concat([df, category], axis=1)
df_new = df_new.drop(columns='genre')
df_new

,ringkasan_sinopsis,Drama,Horor,Komedi,Laga,Romantis
0,Raden Mas Said putra sulung Tumenggung Wilarik...,1,0,0,0,0
1,Soe Hok Gie adalah seorang aktivis yang hidup ...,1,0,0,0,0
2,Guru Bangsa Tjokroaminoto menceritakan tentang...,1,0,0,0,0
3,POL menceritakan kisah hidup yang luar biasa d...,1,0,0,0,0
4,Perjalanan pahlawan Indonesia KH Ahmad Dahlan ...,1,0,0,0,0
...,...,...,...,...,...,...
1000,Winter in Tokyo berpusat pada kehidupan Ishida...,0,0,0,0,1
1001,Markonah melarikan diri ke Jakarta karena akan...,0,0,0,0,1
1002,"Tempat aking lebih dari 36 jam, Last Night ada...",0,0,0,0,1
1003,Proyek baru ini adalah tentang seorang lelaki ...,0,0,0,0,1


In [4]:
sinopsis = df_new['ringkasan_sinopsis'].values
label = df_new[['Drama', 'Horor','Komedi', 'Laga', 'Romantis']]

In [5]:
from sklearn.model_selection import train_test_split
sinopsis_train, sinopsis_test, label_train, label_test = train_test_split(sinopsis, label, test_size=0.2)

In [6]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=5000, oov_token='x')
tokenizer.fit_on_texts(sinopsis_train)
tokenizer.fit_on_texts(sinopsis_test)

sequence_train = tokenizer.texts_to_sequences(sinopsis_train)
sequence_test = tokenizer.texts_to_sequences(sinopsis_test)

padded_train = pad_sequences(sequence_train)
padded_test = pad_sequences(sequence_test)

In [7]:
import tensorflow as tf
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=5000, output_dim=16),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(5, activation='softmax'),
])
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [8]:
class myCallback(tf.keras.callbacks.Callback) :
  def on_epoch_end(self, epoch, logs={}) :
    if(logs.get('accuracy') > 0.9) :
      self.model.stop_training = True
callbacks = myCallback()

In [9]:
num_epochs = 30
history = model.fit(padded_train, label_train, epochs=num_epochs, 
                    validation_data=(padded_test, label_test), verbose=2, callbacks=callbacks)

Epoch 1/30
26/26 - 17s - loss: 1.6094 - accuracy: 0.1940 - val_loss: 1.6156 - val_accuracy: 0.1244
Epoch 2/30
26/26 - 14s - loss: 1.6033 - accuracy: 0.2201 - val_loss: 1.6258 - val_accuracy: 0.1393
Epoch 3/30
26/26 - 14s - loss: 1.5785 - accuracy: 0.3495 - val_loss: 1.6054 - val_accuracy: 0.1791
Epoch 4/30
26/26 - 14s - loss: 1.3713 - accuracy: 0.4204 - val_loss: 1.6512 - val_accuracy: 0.2488
Epoch 5/30
26/26 - 14s - loss: 1.1299 - accuracy: 0.4614 - val_loss: 1.7530 - val_accuracy: 0.2040
Epoch 6/30
26/26 - 14s - loss: 0.9818 - accuracy: 0.5311 - val_loss: 1.9471 - val_accuracy: 0.2836
Epoch 7/30
26/26 - 14s - loss: 0.7490 - accuracy: 0.6866 - val_loss: 2.3522 - val_accuracy: 0.2537
Epoch 8/30
26/26 - 14s - loss: 0.4924 - accuracy: 0.8383 - val_loss: 3.3930 - val_accuracy: 0.2736
Epoch 9/30
26/26 - 14s - loss: 0.3010 - accuracy: 0.9167 - val_loss: 3.8444 - val_accuracy: 0.3184
